<h2>Basic rigging chat<h2>

In [1]:
import rigging as rg

generator = rg.get_generator("gpt-4o-mini")
chat = await generator.chat(
    [
        {"role": "system", "content": "You're a helpful assistant."},
        {"role": "user", "content": "Say hello!"},
    ]
).run()
print(chat.last)
#print(chat.conversation)

[assistant]: Hello! How can I assist you today?


<h2>Templating<h2>

In [2]:
import rigging as rg

template = (
    rg.get_generator("gpt-4o-mini")
    .chat("What is the capitol of $country?")
)

for country in ["France", "Germany"]:
    chat = await template.apply(country=country).run()
    print(chat.last)

[assistant]: The capital of France is Paris.
[assistant]: The capital of Germany is Berlin.


<h2>Parsing<h2>

In [3]:
import rigging as rg
from pydantic import StringConstraints
from typing import Annotated

str_strip = Annotated[str, StringConstraints(strip_whitespace=True)]

class Summary(rg.Model):
    content: str_strip

message = rg.Message(
    "assistant",
    "Sure, the summary is: <summary  > Rigging is a very powerful library </summary>. I hope that helps!"
)

print(message.parse(Summary))

print(message)
#print(message.content)
#print(message.parse(Summary))

<summary>Rigging is a very powerful library</summary>
[assistant]: Sure, the summary is: <summary>Rigging is a very powerful library</summary>. I hope that helps!


<h2>XML-Tags<h2>

In [4]:
class Martin(rg.Model):
    content: str

def involves_Bismarck(message: rg.Message) -> tuple[bool, list[rg.Message]]:
    if "bismarck" not in message.content.lower():
        return True, [message, rg.Message("user", "Please include Otto van Bismarck")] # (1)!
    return False, [message]

chat = (
    await
    rg.get_generator("gpt-3.5-turbo")
    .chat(f"Tell me the name of great German wanderers between {Martin.xml_tags()} tags.")
    .until_parsed_as(Martin)
    .until(involves_Bismarck, drop_dialog=False)
    .run()
)

print(chat.conversation)

[user]: Tell me the name of great German wanderers between <martin></martin> tags.

[assistant]: <martin>Wolfgang von Goethe, Johannes Gutenberg, Heinrich Heine, Ludwig van Beethoven</martin>

[user]: Please include Otto van Bismarck

[assistant]: <martin>Wolfgang von Goethe, Johannes Gutenberg, Heinrich Heine, Ludwig van Beethoven, Otto von Bismarck</martin>


<h2>Tools<h2>

In [5]:
from typing import Annotated
import requests
import rigging as rg

class WeatherTool(rg.Tool):
   name = "weather"
   description = "A tool to get the weather for a location"

   def get_for_city(self, city: Annotated[str, "The city name to get weather for"]) -> str:
      try:
            city = city.replace(" ", "+")
            return requests.get(f"http://wttr.in/{city}?format=2").text
      except:
            return "Failed to call the API"

chat = (
   await 
   rg.get_generator("gpt-4o-mini")
   .chat("How is the weather in london?")
   .using(WeatherTool())
   .run()
)
#print(chat.conversation)
print(chat.last)

[assistant]: The weather in London is currently sunny with a temperature of 14°C. The wind is blowing from the southwest at 18 km/h.


In [6]:
chat = (
   await 
   rg.get_generator("gpt-4o-mini")
   .chat("Who played Harry Potter?")
   .using(WeatherTool())
   .run()
)
#print(chat.conversation)
print(chat.last)

[assistant]: Daniel Radcliffe played the role of Harry Potter in the film series based on J.K. Rowling's books. He starred in all eight movies from "Harry Potter and the Sorcerer's Stone" (also known as "Harry Potter and the Philosopher's Stone") to "Harry Potter and the Deathly Hallows – Part 2."


<h2>Tool Combinations<h2>

In [7]:
from typing import Annotated
import requests
import rigging as rg
import os

api_key = os.getenv("GEOCODE_API_KEY")

class WeatherTool(rg.Tool):
   name = "weather"
   description = "A tool to get the weather for a location"

   def get_for_city(self, city: Annotated[str, "The city name to get weather for"]) -> str:
      try:
            city = city.replace(" ", "+")
            return requests.get(f"http://wttr.in/{city}?format=2").text
      except:
            return "Failed to call the API"


class GeoTool(rg.Tool):
    name = "Geolocation"
    description = "Given latitude and longitude, call the Google Geocoding API to retrieve a formatted address."

    def get_for_city(self, latitude: Annotated[str, "The latitude to get the adress for"], longitude: Annotated[str, "The longitude to get the adress for"])  -> str:
        try:
            url = "https://maps.googleapis.com/maps/api/geocode/json"
            params = {
                "latlng": f"{latitude},{longitude}",
                "key": api_key
            }
            response = requests.get(url, params=params)
            data = response.json()
            return data["results"][0].get("formatted_address")
        except:
            return "Failed to call the API"
chat = (
   await 
   rg.get_generator("gpt-4")
   .chat("What is the weather at latitude 40.758896 and longitude -73.985130")
   .using(WeatherTool(),GeoTool())
   .run()
)
print(chat.conversation)
#print(chat.last)


[system]: # Tool Use
In this environment you have access to a set of tools you can use to improve your responses.

## Tool Call Format
<tool_calls>
  <call tool="$TOOL_A" function="$FUNCTION_A">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
  <call tool="$TOOL_B" function="$FUNCTION_B">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
</tool_calls>

## Available Tools
<tools>
  <tool name="weather" description="A tool to get the weather for a location">
    <functions>
      <function name="get_for_city" description="">
        <parameters>
          <parameter name="city" type="str" description="The city name to get weather for"></parameter>
        </parameters>
      </function>
    </functions>
  </tool>
  <tool name="Geolocation" description="Given latitude and longitude, call the Google Geocoding API to retrieve a formatted address.">
    <functions>
      <function name="get_for_city" description="">
        <parameters>

<h2>Tool Combination Management<h2>

In [8]:
import rigging as rg
import os
import io
import contextlib
from pydantic import StringConstraints
from typing import Annotated

class PythonExecTool(rg.Tool):
    name = "python_exec"
    description = "Use this tool to run Python code for example to open files and get their contents. Must use print(...) to return results and python code all in one line. Print for results to be returned"

    def run(self, code: Annotated[str, "Python code to execute"]) -> str:
        buf = io.StringIO()
        with contextlib.redirect_stdout(buf):
            try:
                exec(code, {}, {})
            except Exception as ex:
                print(f"[ERROR] Exception in user code: {ex}")
        output = buf.getvalue().strip()
        return output if output else "[No output from code execution]"

In [14]:
chat = (
   await 
   rg.get_generator("gpt-4")
   .chat("What is the weather at the location from file ./Location.txt?")
   .using(WeatherTool(),GeoTool(), PythonExecTool())
   .run()
)
#print(chat.conversation)
print(chat.conversation)

[system]: # Tool Use
In this environment you have access to a set of tools you can use to improve your responses.

## Tool Call Format
<tool_calls>
  <call tool="$TOOL_A" function="$FUNCTION_A">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
  <call tool="$TOOL_B" function="$FUNCTION_B">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
</tool_calls>

## Available Tools
<tools>
  <tool name="weather" description="A tool to get the weather for a location">
    <functions>
      <function name="get_for_city" description="">
        <parameters>
          <parameter name="city" type="str" description="The city name to get weather for"></parameter>
        </parameters>
      </function>
    </functions>
  </tool>
  <tool name="Geolocation" description="Given latitude and longitude, call the Google Geocoding API to retrieve a formatted address.">
    <functions>
      <function name="get_for_city" description="">
        <parameters>

In [25]:
chat = (
   await 
   rg.get_generator("gpt-4o")
   .chat("What is the weather at the location from the file ./Location.txt?")
   .using(WeatherTool(),GeoTool(), PythonExecTool())
   .run()
)
#print(chat.conversation)
print(chat.conversation)

[system]: # Tool Use
In this environment you have access to a set of tools you can use to improve your responses.

## Tool Call Format
<tool_calls>
  <call tool="$TOOL_A" function="$FUNCTION_A">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
  <call tool="$TOOL_B" function="$FUNCTION_B">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
</tool_calls>

## Available Tools
<tools>
  <tool name="weather" description="A tool to get the weather for a location">
    <functions>
      <function name="get_for_city" description="">
        <parameters>
          <parameter name="city" type="str" description="The city name to get weather for"></parameter>
        </parameters>
      </function>
    </functions>
  </tool>
  <tool name="Geolocation" description="Given latitude and longitude, call the Google Geocoding API to retrieve a formatted address.">
    <functions>
      <function name="get_for_city" description="">
        <parameters>

In [24]:
def log_attempts(message: rg.Message) -> tuple[bool, list[rg.Message]]:
    print(f"Attempt: {message.content}")
    return False, [message]

class Weather(rg.Model):
    content: str

chat = (
   await 
   rg.get_generator("gpt-4o")
   .chat(f"What is the weather at the location from file ./Location.txt? Please place the weather conditions between {Weather.xml_tags()} tags")
   .using(WeatherTool(),GeoTool(), PythonExecTool())
    .until(log_attempts, drop_dialog=False)
    .until_parsed_as(Weather,max_rounds=5, attempt_recovery=True, drop_dialog=False)
   .run()
)
#print(chat.conversation)
print(chat.conversation)


Attempt: To provide you with the weather information for the location specified in the file, I'll first need to read the contents of `./Location.txt` to determine the location details.
[system]: # Tool Use
In this environment you have access to a set of tools you can use to improve your responses.

## Tool Call Format
<tool_calls>
  <call tool="$TOOL_A" function="$FUNCTION_A">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
  <call tool="$TOOL_B" function="$FUNCTION_B">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
</tool_calls>

## Available Tools
<tools>
  <tool name="weather" description="A tool to get the weather for a location">
    <functions>
      <function name="get_for_city" description="">
        <parameters>
          <parameter name="city" type="str" description="The city name to get weather for"></parameter>
        </parameters>
      </function>
    </functions>
  </tool>
  <tool name="Geolocation" description

In [23]:
chat = (
   await 
   rg.get_generator("gpt-4o")
   .chat(f"What is the weather at the location from file ./Location.txt? Please place the weather conditions between {Weather.xml_tags()} tags and only the weather conditions. Never use the tags otherwise. You must wrap any result you want returned from the python code in a print statement.")
   .using(WeatherTool(),GeoTool(), PythonExecTool())
    .until_parsed_as(Weather,max_rounds=10, attempt_recovery=True, drop_dialog=False)
   .run()
)
print(chat.conversation)

[system]: # Tool Use
In this environment you have access to a set of tools you can use to improve your responses.

## Tool Call Format
<tool_calls>
  <call tool="$TOOL_A" function="$FUNCTION_A">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
  <call tool="$TOOL_B" function="$FUNCTION_B">
    <parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
  </call>
</tool_calls>

## Available Tools
<tools>
  <tool name="weather" description="A tool to get the weather for a location">
    <functions>
      <function name="get_for_city" description="">
        <parameters>
          <parameter name="city" type="str" description="The city name to get weather for"></parameter>
        </parameters>
      </function>
    </functions>
  </tool>
  <tool name="Geolocation" description="Given latitude and longitude, call the Google Geocoding API to retrieve a formatted address.">
    <functions>
      <function name="get_for_city" description="">
        <parameters>